# Convert documents to Excel sheet
- Collect list of Word documents
- For each Word document:
1) Extract all text
2) Combine all text to a single string (remove excess whitespace first)
3) Break by keywords STUDENT and CHATGPT
4) Return dataframe with single row per student response
5) Run word count on student and ChatGPT responses
- Combine all dataframes

In [2]:
import re
import pandas as pd
from docx import Document
import os
import glob

In [3]:
replacement_list=[
    {'Html': '\n', 'Replace with': ''},
    {'Html': '<strong>', 'Replace with': ''},
    {'Html': '</strong>', 'Replace with': ''}, 
    {'Html': '<em>', 'Replace with': ''},
    {'Html': '</em>', 'Replace with': ''},
    {'Html': '&nbsp;', 'Replace with': ' '}, 
    {'Html': '<div>', 'Replace with': ''},
    {'Html': '<div\sclass=.+>', 'Replace with': ''},
    {'Html': '<span\sclass=.+><span\sdata-offset-key.+>', 'Replace with': ''},
    {'Html': '<span\sdata-offset-key.+>', 'Replace with': ''},
    {'Html': '<span\sstyle=.+>', 'Replace with': ''},
    {'Html': '</div>', 'Replace with': ''},
    #{'Html': '<img.+>', 'Replace with': '[image]'}, Removes equations. 
    {'Html': 'https://.+\s', 'Replace with': '[url]'},
    {'Html': '&amp;', 'Replace with': '&'},
    {'Html': '<script.+</script>', 'Replace with': ''},
    {'Html': '<ol>', 'Replace with': ''},
    {'Html': '</ol>', 'Replace with': ''},
    {'Html': '<ul>', 'Replace with': ''},
    {'Html': '</ul>', 'Replace with': ''},
    {'Html': '<p>', 'Replace with': ''}, 
    {'Html': '</p>', 'Replace with': ' '}, 
    {'Html': '<li>', 'Replace with': ''}, # Special case 
    {'Html': '<sup>', 'Replace with': '^'}, 
    {'Html': '</sup>', 'Replace with': ''},
    {'Html': '\\n', 'Replace with': ''},
    {'Html': '\\\\:', 'Replace with': ''},
]

def extract_equation(entry):
    match_list=re.findall('<p>(.*)</p>', entry)
    extract_equation_list=[]
    equation_regex='<img\sclass=.equation_image.+?>'
    for sentence in match_list:
        if re.findall(equation_regex, sentence)!=[]:
            split_without_equation=re.split(equation_regex, sentence)
            raw_equation_list=re.findall(equation_regex, sentence)
            clean_equation_list=[]
            for raw_equation in raw_equation_list:
                # First try to extact the exact equation display
                try:
                    math_to_display=re.search('data-equation-content=".*?"', raw_equation).group()
                    cleaned_math=re.sub('data-equation-content="', '', math_to_display)
                    cleaned_math=re.sub('"', '', cleaned_math)
                    clean_equation_list.append(cleaned_math)
                # If an error occurs (such as not finding data-equation-content) substitute for a generic statement
                except:
                    clean_equation_list.append('[equation_image]')
            clean_sentence=split_without_equation.pop(0)
            while len(clean_equation_list)!=0:
                clean_sentence+=clean_equation_list.pop(0)
                clean_sentence+=split_without_equation.pop(0)
            extract_equation_list.append(clean_sentence)
        else:
            extract_equation_list.append(sentence)
    final_string = ' '.join(extract_equation_list)
    final_string = re.sub('&nbsp;', '', final_string)
    return final_string

<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:34: SyntaxWarning: invalid escape sequence '\s'
<>:9: SyntaxWarning: invalid escape sequence '\s'
<>:10: SyntaxWarning: invalid escape sequence '\s'
<>:11: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:15: SyntaxWarning: invalid escape sequence '\s'
<>:34: SyntaxWarning: invalid escape sequence '\s'
C:\Users\dcham\AppData\Local\Temp\ipykernel_24596\3777762459.py:9: SyntaxWarning: invalid escape sequence '\s'
  {'Html': '<div\sclass=.+>', 'Replace with': ''},
C:\Users\dcham\AppData\Local\Temp\ipykernel_24596\3777762459.py:10: SyntaxWarning: invalid escape sequence '\s'
  {'Html': '<span\sclass=.+><span\sdata-offset-key.+>', 'Replace with': ''},
C:\Users\dcham\AppData\Local\Temp\ipyke

In [4]:
def extract_text_from_docx(document_opened):
    # Extract text from document into list of paragraphs
    list_of_paragraphs = []
    for para in document_opened.paragraphs:
        list_of_paragraphs.append(para.text)
    # Combine into single string
    single_string = ' '.join(list_of_paragraphs)
    # Split by STUDENT and CHATGPT
    full_split = re.split('STUDENT | CHATGPT', single_string)
    clean_split = [string for string in full_split if len(string) > 0]
    # Create list of dicts for each response
    list_of_response_dicts = []
    while len(clean_split) > 1: 
        temp_dict = {
             "Student Response": clean_split.pop(0),
             "ChatGPT Feedback": clean_split.pop(0)
        }
        list_of_response_dicts.append(temp_dict)
    # Create Dataframe from list
    df_responses = pd.DataFrame(list_of_response_dicts)
    return df_responses

def clean_raw_html(entry):
    revised_entry = extract_equation(entry)
    for replacement_dict in replacement_list:
        revised_entry = re.sub(replacement_dict['Html'], replacement_dict['Replace with'], revised_entry)
    return revised_entry

def extract_text_from_xlsx(document_opened):
    df = pd.read_excel(document_opened, skiprows=1)
    df['Student Response'] = df['Raw HTML Response'].apply(clean_raw_html)
    df['ChatGPT Feedback'] = df['ChatGPT Raw Feedback'].apply(clean_raw_html)
    return df

def define_sentence_length(list_of_substrings):
    if type(list_of_substrings) != type([]): # Returns 0 for nan values, which str.split() returns for empty cells
        return 0
    else:
        return len(list_of_substrings)

In [12]:
base_dir=os.getcwd()
if os.name == 'nt':
    path_break='\\'
else:
    path_break='/'

# Collects names of files in the Raw_Files folders. 
all_docx_files = []
all_xlsx_files = []
for file in glob.glob(f'{base_dir}{path_break}Raw_Files{path_break}*'):
    if '$' not in file:
        if 'xlsx' in file:
            all_xlsx_files.append(file)
        elif 'docx' in file:
            all_docx_files.append(file)

list_of_dfs = []
for file_name in all_docx_files:
    temp_df = extract_text_from_docx(Document(file_name))
    root_name = f'{base_dir}{path_break}Raw_Files{path_break}'
    short_file_name = file_name[len(root_name):]
    list_of_info = re.split('-', short_file_name)

    temp_df['Instructor'] = list_of_info[0].strip()
    temp_df['Course'] = list_of_info[1].strip()
    temp_df['Response Word Count'] = temp_df['Student Response'].str.split().map(define_sentence_length) 

    list_of_dfs.append(temp_df)

for file_name in all_xlsx_files:
    temp_df = extract_text_from_xlsx(file_name)
    root_name = f'{base_dir}{path_break}Raw_Files{path_break}'
    short_file_name = file_name[len(root_name):]
    list_of_info = re.split('-', short_file_name)

    temp_df['Instructor'] = list_of_info[0].strip()
    temp_df['Course'] = list_of_info[1].strip()
    temp_df['Response Word Count'] = temp_df['Student Response'].str.split().map(define_sentence_length)
    
    list_of_dfs.append(temp_df)

final_df = pd.concat(list_of_dfs)
final_df.to_excel('combined_responses.xlsx')

In [ ]:
test_df = pd.read_excel('./Raw_Files/Darryl - MATH 111 - Processed.xlsx', skiprows=1)
test_df['clean_responses'] = test_df['Raw HTML Response'].apply(clean_raw_html)
test_df.to_excel('check_test.xlsx')

#### Note: ChatGPT scores and whether the instructor re-prompted ChatGPT are manually added to "combined_responses" 